In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import Constants

In [2]:
box_scores = os.listdir(Constants.SCORE_DIR)

In [4]:
box_scores = [os.path.join(Constants.SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [6]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    
    return soup

In [9]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "Team"
    cols[-1] = "Total"
    line_score.columns = cols
    
    line_score = line_score[["Team", "Total"]]
    return line_score

In [11]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [14]:
box_score = box_scores[0]
soup = parse_html(box_score)
line_score = read_line_score(soup)
teams = list(line_score["Team"])

summaries = []
for team in teams:
    basic = read_stats(soup, team, "basic")
    advanced = read_stats(soup, team, "advanced")
    
    total = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])


/var/folders/t2/p3yp6v4544jds05t77dbbtq40000gp/T/ipykernel_21397/1869830524.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
/var/folders/t2/p3yp6v4544jds05t77dbbtq40000gp/T/ipykernel_21397/385819283.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
/var/folders/t2/p3yp6v4544jds05t77dbbtq40000gp/T/ipykernel_21397/385819283.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
/va